In [41]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import cv2  
import sys
import math
from matplotlib import pyplot as plt
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
import pandas as pd
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input

In [42]:
# Create ResNet50 model with weights of imagenet
model = ResNet50(weights='imagenet', include_top=False)
model.summary()

94773248/94765736 [==============================] - 133s 1us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
________________________

In [43]:
# Load training images from folder and use ResNet50 to extract features
rootpath = './train/'
training_emb = []
for root, dirs, files in os.walk(rootpath):
    for file in files:
        img = image.load_img(os.path.join(rootpath, file), target_size=(224, 224))
        img_tensor = image.img_to_array(img)
        img_tensor = np.expand_dims(img_tensor, axis=0)
        img_tensor = preprocess_input(img_tensor)
        emb = model.predict(img_tensor)
        emb = emb.flatten()
        training_emb.append(emb)

In [45]:
# Get training locations from csv file
nd = np.genfromtxt("train.csv", str, delimiter = ',', skip_header=True)
training_pos = nd[:, 1:3]
training_pos = training_pos.astype(float)


In [47]:
# Load test images from folder and use ResNet50 to extract features
testpath = './test/'
test_emb = []
test_name = []
for root, dirs, files in os.walk(testpath):
    for file in files:
        test_name.append(file)
        img = image.load_img(os.path.join(testpath, file), target_size=(224, 224))
        img_tensor = image.img_to_array(img)
        img_tensor = np.expand_dims(img_tensor, axis=0)
        img_tensor = preprocess_input(img_tensor)
        emb = model.predict(img_tensor)
        emb = emb.flatten()
        test_emb.append(emb)

In [48]:
def cosine_similarity(x, y):
    num = x.dot(y.T)
    denom = np.linalg.norm(x) * np.linalg.norm(y)
    return num/denom

In [49]:
def euclidean_distance(x, y):
    return np.linalg.norm(x - y)

In [50]:
# Get similarities between each training and test image
sim_img = []
for i in range(len(test_emb)):
    most_sim = []
    for j in range(len(training_emb)):
        sim = cosine_similarity(training_emb[j], test_emb[i])
        most_sim.append([sim, j])
    most_sim.sort(reverse=True)
    sim_index = []
    for k in range(100):
        sim_index.append(most_sim[k][1])
    sim_img.append(sim_index)

In [51]:
sim_img_df = pd.DataFrame(sim_img)
sim_img_df.to_csv('sim_img.csv', header=0, index=0)

In [66]:
# Use SIFT to extract features of images and find 'good matching' key points
emp = 0
final_sim_img = []
for r, d, fs in os.walk(testpath):
    testi = 0
    for f in fs:
        img1 = cv2.imread(os.path.join(testpath, f))
        sift = cv2.xfeatures2d.SIFT_create()
        kp1, des1 = sift.detectAndCompute(img1, None)

        ratio = 0.5
        sift_sim = []
        for i in sim_img[testi][:20]:
            for root, dirs, files in os.walk(rootpath):
                j = 0
                for file in files:
                    if j == i:
                        img2 = cv2.imread(os.path.join(rootpath, file))
                        break
                    j += 1
            kp2, des2 = sift.detectAndCompute(img2, None)
            kp_img2 = cv2.drawKeypoints(img2, kp2, None)

            matcher = cv2.BFMatcher()
                
            try:
                raw_matches = matcher.knnMatch(des1, des2, k = 2)
            except:
                pass
            
            good_matches = []
            for index, pair in enumerate(raw_matches):
                try:
                    m1, m2 = pair
                    if m1.distance < ratio * m2.distance:
                        good_matches.append([m1])
                except ValueError:
                    pass
            
            if len(good_matches) > 0:
                #ave_dis = 0
                #for m in good_matches:
                #    ave_dis += m[0].distance
                #ave_dis = ave_dis / len(good_matches)
                #sift_sim.append([ave_dis, i])
                sift_sim.append([len(good_matches), i])
        
        if(len(sift_sim) > 0):
            sift_sim.sort(reverse=True)
            final_sim_img.append(sift_sim[0][1])
        else:
            final_sim_img.append(sim_img[testi][0])
            emp += 1
        testi += 1
print(emp)

33


In [62]:
# Consider the most similarity image's location as the predicted location of the test image
test_pos = training_pos[final_sim_img]
t_name = np.array(test_name)
t_name = t_name.reshape(-1, 1)
for i in range(len(t_name)):
    t_name[i][0] = t_name[i][0][:-4]

array([['IMG4287_3'],
       ['IMG4288_5'],
       ['IMG4289_5'],
       ...,
       ['IMG5484_4'],
       ['IMG5485_3'],
       ['IMG5486_2']], dtype='<U13')

In [71]:
output = np.concatenate((t_name, test_pos), axis=1)
output_df = pd.DataFrame(output, columns=['id', 'x', 'y'])
output_df.to_csv('output.csv', index=0)